In [ ]:
import re
import json
import csv

loan_records = []
person_records = []

# Load entire file as a string
with open("/content/noisy_data+1.txt", "r", encoding="utf-8") as f:
    data = f.read()

# Extract loan JSON objects using regex
json_loans = re.findall(r'{\s*"loan_id"\s*:\s*"LN\d+".*?}', data)

for item in json_loans:
    try:
        loan = json.loads(item)
        loan_records.append(loan)
    except json.JSONDecodeError:
        continue

# Extract personal records line by line
for line in data.splitlines():
    line = line.strip()
    if line.startswith("name,"):
        parts = line.split(",")
        try:
            person = {
                "person_id": f"P{len(person_records)+1:06}",  # Unique ID
                "name": parts[1],
                "gender": parts[3],
                "address": parts[5],
                "pan": parts[7].replace("pan", "").strip(),
                "dob": parts[9].replace("dob", "").strip()
            }
            person_records.append(person)
        except IndexError:
            continue

# ✅ Save personal records
if person_records:
    with open("personal_data.csv", "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=person_records[0].keys())
        writer.writeheader()
        writer.writerows(person_records)
    print("✅ Saved personal_data.csv")

# ✅ Save loan records (add foreign key reference)
if loan_records:
    for i in range(min(len(loan_records), len(person_records))):
        loan_records[i]["person_id"] = person_records[i]["person_id"]

    with open("loan_data.csv", "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=loan_records[0].keys())
        writer.writeheader()
        writer.writerows(loan_records)
    print("✅ Saved loan_data.csv")


✅ Saved personal_data.csv
✅ Saved loan_data.csv


In [ ]:
import pandas as pd

# Load both datasets
df_loan = pd.read_csv("loan_data.csv")
df_personal = pd.read_csv("personal_data.csv")

# Merge on person_id
df_merged = df_loan.merge(df_personal, on="person_id", how="inner")

# Save merged data
df_merged.to_csv("merged_loan_person_data.csv", index=False)

print("✅ Merged data saved to merged_loan_person_data.csv")
print("🧾 Final shape:", df_merged.shape)


✅ Merged data saved to merged_loan_person_data.csv
🧾 Final shape: (42102, 12)
